In [66]:
import catboost as cb
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
labels = pd.read_csv('labels_training_set.csv')

In [137]:
processed = pd.read_csv('features_01.csv')

In [121]:
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [135]:
features.head()

,person,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel
0,0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,14,6.0,Referral
1,00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,0,16.0,Direct
2,00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,66,10.0,Paid
3,000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,5,20.6,Direct
4,000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,2,17.0,Paid


In [147]:
features = processed.set_index('person').drop(['primer_visita','ultima_visita'],axis=1)

In [148]:
dataset = labels.set_index('person').join(features,how='left')

In [149]:
dataset.head()

,label,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel
person,,,,,,,,,,,,
0566e9c1,0,viewed product,iPhone 5s,Smartphone,1,17,23,68,9,0,7.555556,Referral
6ec7ee77,0,visited site,Ninguno,Smartphone,0,1,0,2,1,3,2.000000,Paid
abe7a2fb,0,viewed product,iPhone 6,Smartphone,0,22,31,96,109,2,0.880734,Referral
34728364,0,viewed product,Lenovo Vibe K5,Smartphone,0,4,24,37,9,4,4.111111,Referral
87ed62de,0,viewed product,Samsung Galaxy J7,Computer,0,1,9,17,1,12,17.000000,Paid


In [150]:
training_features = dataset.drop(['label'],axis=1)
training_labels = dataset['label']

In [151]:
xtrain,xtest,ytrain,ytest = train_test_split(training_features,training_labels)

In [152]:
catBoost = cb.CatBoostClassifier(iterations=200,
                                 depth=8,
                                 l2_leaf_reg=50,
                                 random_strength=0.1,
                                 bagging_temperature=0,
                                 border_count=128)

In [153]:
catBoost.fit(xtrain,ytrain,verbose=0,cat_features=[0,1,2,10])

In [154]:
roc_auc_score(ytest,catBoost.predict_proba(xtest)[:,1])

0.7545946774229066

In [111]:
prediction_features = kaggle.set_index('person').join(features,how='left')

In [115]:
pred = catBoost.predict_proba(prediction_features)[:,1]
submit = kaggle.set_index('person')
submit['label'] = pred

In [116]:
submit.head()

,label
person,
4886f805,0.017131
0297fc1e,0.163832
2d681dd8,0.042694
cccea85e,0.062610
4c8a8b93,0.023112


In [117]:
#submit.to_csv('submit.csv')